# PARTE B: Uso de expresiones regulares para identificar patrones

## Lectura del archivo "gene.fna"

In [2]:
#!pip install biopython 
#instalamos en caso indica que la biblioteca Biopython no está instalada en tu entorno de Jupyter Notebook

In [3]:
import os
from Bio import SeqIO

# Obtener el directorio actual del notebook 
directorio = os.path.dirname(os.path.abspath('__file__'))

# Construir la ruta relativa al archivo .fna (esto se debe a que los directorios de cada integrantes pueden ser diferentes,
# entonces es mejor construir una ruta relativa)
file_ruta = os.path.join(directorio, "ncbi_dataset", "data", "gene.fna")

# Verificar si el archivo existe
"""
if os.path.exists(file_ruta):
    print("El archivo existe.")
else:
    print("El archivo no existe.")
"""
# Leyendo el archivo tipo .fna
with open(file_ruta, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        print(f"{record.description}")
        print(f"{record.seq}\n")


NC_000005.10:c149551439-149492982 CSNK1A1 [organism=Homo sapiens] [GeneID=1452] [chromosome=5]
...

NC_060929.1:c150086185-150027795 CSNK1A1 [organism=Homo sapiens] [GeneID=1452] [chromosome=5]
...



### a) El porcentaje de nucleòtidos C o G en toda la secuencia.

In [12]:
import re

with open(file_ruta, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        #print(f"{record.description}")
        secuencia = str(record.seq)
        #print(f"{record.seq}\n")
# Contar nucleótidos C y G en la secuencia
conteo_C = len(re.findall('C', secuencia))
conteo_G = len(re.findall('G', secuencia))

# Calcular el total de nucleótidos
total_nucleotidos = len(secuencia)

# Calcular el porcentaje de C y G
porcentaje_CG = ((conteo_C + conteo_G) / total_nucleotidos) * 100

print(f"El porcentaje de nucleótidos C o G en la secuencia es: {porcentaje_CG:.2f}%")


El porcentaje de nucleótidos C o G en la secuencia es: 39.28%


### b) Tomando secuencias de tres nucleóticos de ventana determine el número de CCG o GCC que se encuentran en la secuencia.

In [10]:
import re

CCG = re.findall(r"CCG", secuencia) 
GCC = re.findall(r"GCC", secuencia)
total_secuencias = len(CCG) + len(GCC) # número total de secuencias CCG o GCC
print("Se encontraron",len(CCG), "coincidencias de CCG.")
print("Se encontraron",len(GCC), "coincidencias de GCC.")
print("El total de coincidencias CCG o GCC es:",total_secuencias)
        

Se encontraron 166 coincidencias de CCG.
Se encontraron 696 coincidencias de GCC.
El total de coincidencias CCG o GCC es: 862


### c) Buscar y contar las secuencias: GAATTC

In [13]:
import re

contador = len(re.findall(r"GAATTC", secuencia))

print("Número de ocurrencias de 'GAATTC':", contador)

Número de ocurrencias de 'GAATTC': 16


### d)¿Es posible automatizar la descarga del archivo directamente desde Python?¿què librerìas y/o sentencias utilizarìa?. Escriba còdigo para dicha descarga
- Para este caso se utilizo la libreria Biopython que es una biblioteca de herramientas y módulos en Python diseñada específicamente para computación bioinformática. Proporciona funcionalidad para manejar secuencias biológicas, realizar búsquedas en bases de datos biológicas, y manipular archivos biológicos en diversos formatos (como FASTA, GenBank, etc.).
- La función "Entrez" nos permite establecer una conexión con la base de datos NCBI, además nos permite realizar la busqueda, descarga y obtener información detallada sobre un registro especifico.
- Tambien se utiliza la sentencia "SeqIO" que proporciona interfaces para leer y escribir secuencias biológicas en diferentes formatos de archivos (FASTA, genbank, etc).

In [4]:
from Bio import Entrez, SeqIO

# Establecer el correo electrónico (NCBI requiere que proporciones un correo electrónico)
Entrez.email = "20220767@lamolina.edu.pe"

# ID del gen que deseas buscar
gene_id = "1452"

# Buscar secuencias de ADN asociadas al gen
handle = Entrez.esearch(db="nucleotide", term=f"{gene_id}[Gene ID]", retmax=10)
record = Entrez.read(handle)
handle.close()

# Obtener los identificadores de las secuencias
id_list = record["IdList"]
print(f"IDs de secuencias encontrados: {id_list}")

# Descargar las secuencias en formato FASTA
ids = ",".join(id_list)
handle = Entrez.efetch(db="nucleotide", id=ids, rettype="fasta", retmode="text")
sequences = handle.read()
handle.close()

# Guardar las secuencias en un archivo FASTA
with open("gene_1452_sequences.fasta", "w") as out_file:
    out_file.write(sequences)
print("Secuencias descargadas y guardadas en gene_1452_sequences.fasta")

# Leer y mostrar un resumen de las secuencias en formato FASTA
for record in SeqIO.parse("gene_1452_sequences.fasta", "fasta"):
    print(f">{record.id}")
    print(record.description)
    print(f"Longitud de la secuencia: {len(record.seq)} nucleótidos")
    print("Secuencia guardada en gene_1452_sequences.fasta")

    
#Esperar un momento a que se descarguen los datos en "gene_1452_sequences.fasta", esto puede tardar unos minutos.
#Este metodo descarga los datos solicitados, esto evita que jupyter notebook reciba demasiada información en un corto
#periodo de tiempo lo cual puede ocasionar fallas o problemas de rendimiento.

IDs de secuencias encontrados: ['1677539332', '1677501110', '1677499413', '1677498920', '568815593', '2194974170', '815059033', '815057550', '808568287', '808566770']
Secuencias descargadas y guardadas en gene_1452_sequences.fasta
>NM_001271741.2
NM_001271741.2 Homo sapiens casein kinase 1 alpha 1 (CSNK1A1), transcript variant 3, mRNA
Longitud de la secuencia: 5324 nucleótidos
Secuencia guardada en gene_1452_sequences.fasta
>NM_001892.6
NM_001892.6 Homo sapiens casein kinase 1 alpha 1 (CSNK1A1), transcript variant 2, mRNA
Longitud de la secuencia: 5360 nucleótidos
Secuencia guardada en gene_1452_sequences.fasta
>NM_001271742.2
NM_001271742.2 Homo sapiens casein kinase 1 alpha 1 (CSNK1A1), transcript variant 4, mRNA
Longitud de la secuencia: 5030 nucleótidos
Secuencia guardada en gene_1452_sequences.fasta
>NM_001025105.3
NM_001025105.3 Homo sapiens casein kinase 1 alpha 1 (CSNK1A1), transcript variant 1, mRNA
Longitud de la secuencia: 5444 nucleótidos
Secuencia guardada en gene_1452_seq

In [2]:
#Una vez descargada la secuencia de datos y guardarlas en un archivo llamado "gene_1452_sequences.fasta", podemos utilizar
#el siguiente código para visualla secuencia de genes:

# Leer las secuencias desde el archivo FASTA
with open("gene_1452_sequences.fasta", "r") as in_file:
    for record in SeqIO.parse(in_file, "fasta"):
        print(f">{record.id}")
        print(record.seq)

>NM_001271741.2
CCCAGAGTGCTCTGCGCCGTGAAGAAGCGGCTCCCGGGGACTGGGGGCATTTTGTGTTGGCTGGAGCTGGAGTAACAAGATGGCGTCGTCCGCGGAGTGACAGGGGTCCCTCTGGGCCGGAGCCGGCGGCAGTGGTGGCAGCGGTATCGCCGCCCTAGCTCACCGCGCCCCTTTTCCAGCCCGCGACGTCGCCGCGCAAGCGAGGCAGCGGCGGCCGCCGAGAAACAAGTGGCCCAGCCTGGTAACCGCCGAGAAGCCCTTCACAAACTGCGGCCTGGCAAAAAGAAACCTGACTGAGCGGCGGTGATCAGGTTCCCCTCTGCTGATTCTGGGCCCCGAACCCCGGTAAAGGCCTCCGTGTTCCGTTTCCTGCCGCCCTCCTCCGTAGCCTTGCCTAGTGTAGGAGCCCCGAGGCCTCCGTCCTCTTCCCAGAGGTGTCGGGGCTTGGCCCCAGCCTCCATCTTCGTCTCTCAGGATGGCGAGTAGCAGCGGCTCCAAGGCTGAATTCATTGTCGGAGGGAAATATAAACTGGTACGGAAGATCGGGTCTGGCTCCTTCGGGGACATCTATTTGGCGATCAACATCACCAACGGCGAGGAAGTGGCAGTGAAGCTAGAATCTCAGAAGGCCAGGCATCCCCAGTTGCTGTACGAGAGCAAGCTCTATAAGATTCTTCAAGGTGGGGTTGGCATCCCCCACATACGGTGGTATGGTCAGGAAAAAGACTACAATGTACTAGTCATGGATCTTCTGGGACCTAGCCTCGAAGACCTCTTCAATTTCTGTTCAAGAAGGTTCACAATGAAAACTGTACTTATGTTAGCTGACCAGATGATCAGTAGAATTGAATATGTGCATACAAAGAATTTTATACACAGAGACATTAAACCAGATAACTTCCTAATGGGTATTGGGCGTCACTGTAATAAGTTATTCCTTATTGATTTTGGTTTGGCCAAAAAGTACAGAGACAACAGGACAAG

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

